# Bayesian Losses Module

## Libraries

In [ ]:
import os
import sys
import importlib
from abc import abstractmethod
from typing import Tuple, Any

import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt

## Functions

In [ ]:
def check_parameters():

    print("Check the existence of trainable parameters in the classes...")

    torch_list_parameters = list(torch_module.parameters())
    assert len(torch_list_parameters) != 0, "No parameters availables in TorchTestModule"

    trainable_variables = tf_module.trainable_variables
    assert len(trainable_variables) != 0, "No trainable parameters available in TFTestModule"
    
    print("Test passed!",'\n\n')

In [ ]:
def check_forward_losses():

    print("Check the forward propagation of the loss functions...")
        
    # Input data
    input_data = np.random.randn(1, 10).astype(np.float32)
    y_true = np.random.randn(1, 10).astype(np.float32)
    y_pred = np.random.randn(1, 10).astype(np.float32)

    # PyTorch forward pass
    torch_input = torch.from_numpy(input_data)
    torch_output = torch_module(torch_input)
    torch_kl_divengence_output = torch_kl_divengence(torch_module)
    torch_elbo_loss_output = torch_elbo_loss(torch.from_numpy(y_true), torch.from_numpy(y_pred), torch_module)

    # Tensorflow forward pass
    tf_input = tf.convert_to_tensor(input_data)
    tf_output = tf_module(tf_input)
    tf_kl_divergence_output = tf_kl_divengence(tf_module)
    tf_elbo_loss_output = tf_elbo_loss(tf.convert_to_tensor(y_true), tf.convert_to_tensor(y_pred), tf_module)

    # Assert that the outputs are similar
    print("Torch output:", torch_output)
    print("TensorFlow output:", tf_output)
    print("Outputs are similar:", np.allclose(torch_output.detach().numpy(), tf_output))

    print("Torch KL divergence output:", torch_kl_divengence_output)
    print("TensorFlow KL divergence output:", tf_kl_divergence_output)
    print("KL divergence outputs are similar:", np.allclose(torch_kl_divengence_output.detach().numpy(), tf_kl_divergence_output))

    print("Torch ELBO loss output:", torch_elbo_loss_output)
    print("TensorFlow ELBO loss output:", tf_elbo_loss_output)
    print("ELBO loss outputs are similar:", np.allclose(torch_elbo_loss_output.detach().numpy(), tf_elbo_loss_output))

    print("Test passed!",'\n\n')

In [ ]:
def run_all_tests():
    
    check_parameters()
    check_forward_losses()

## Illia

When the backend is selected we can import illia, if we want to change the backend we need to restart the kernel. The backend can't be changed dynamically.

In [28]:
from illia.torch.nn.losses import (
    KLDivergenceLoss as TorchKLDivergenceLoss, 
    ELBOLoss as TorchELBOLoss 
)
from illia.torch.nn.linear import Linear as TorchLinear

from illia.tf.nn.losses import (
    KLDivergenceLoss as TFKLDivergenceLoss, 
    ELBOLoss as TFELBOLoss
)
from illia.tf.nn.linear import Linear as TFLinear

In [ ]:
import illia
from illia.torch.nn.base import BayesianModule as TorchBayesianModule
from illia.tf.nn.base import BayesianModule as TFBayesianModule

In [29]:
class TorchTestModule(TorchBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TorchLinear(10, 5)

    def forward(self, x):

        return self.linear(x)

    def kl_cost(self):

        return torch.tensor(1.0), 1

In [30]:
class TFTestModule(TFBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TFLinear(10, 5)

    def call(self, x):

        return self.linear(x)

    def kl_cost(self):

        return tf.constant(1.0), 1

Defining the classes

In [31]:
torch_kl_divengence = TorchKLDivergenceLoss()
torch_elbo_loss = TorchELBOLoss(loss_function=torch.nn.MSELoss())
torch_module = TorchTestModule()

In [32]:
tf_kl_divengence = TFKLDivergenceLoss()
tf_elbo_loss = TFELBOLoss(loss_function=tf.keras.losses.MeanSquaredError())
tf_module = TFTestModule()

In [ ]:
run_all_tests()